In [1]:
from urllib.request import urlretrieve
import sys
import pandas as pd
import requests
import math
import zipfile
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


spark = (
    SparkSession.builder.appName("Assignment_2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
pd.options.display.float_format = '{:,.4f}'.format

OUTPUT_DIR = "../data/raw/ABS/"

22/08/31 00:41:43 WARN Utils: Your hostname, Bruce-PC resolves to a loopback address: 127.0.1.1; using 172.21.202.84 instead (on interface eth0)
22/08/31 00:41:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/31 00:41:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def pull_direct(url, output_dir, file_name):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        urlretrieve(url, f"{output_dir}{file_name}")
        print(f"Request succeed: pulling from{url}\nFile saved in: {output_dir}{file_name}")
    except Exception as e:
        print(f"********\nRequest failure: ")
        print(e)
        print("********")

SA2 match tabel

In [40]:
### Pull SA2 match table
url = f"https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/allocation-files/SA2_2021_AUST.xlsx"
file_name = "SA2_2021_AUST.xlsx"
pull_direct(url, OUTPUT_DIR, file_name)

Request succeed: pulling fromhttps://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/allocation-files/SA2_2021_AUST.xlsx
File saved in: ../data/raw/ABS/SA2_2021_AUST.xlsx


SA2 shape file

In [41]:
url = f"https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"
file_name = "SA2_2021_AUST_SHP_GDA2020.zip"
pull_direct(url, OUTPUT_DIR, file_name)

Request succeed: pulling fromhttps://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip
File saved in: ../data/raw/ABS/SA2_2021_AUST_SHP_GDA2020.zip


In [18]:
with zipfile.ZipFile(f"{OUTPUT_DIR}{file_name}", "r") as zip_ref:
    zip_ref.extractall(f"{OUTPUT_DIR}digitalBoundary/")

ABS Data

In [3]:
headers = {
    "accept": "text/csv"
}

Estimated Resident Population (ERP) (2001 to 2021)

In [4]:
headers = {
    "accept": "text/csv"
}
url = "https://api.data.abs.gov.au/data/ABS,ABS_ANNUAL_ERP_ASGS2021,1.2.0/.SA2..A?startPeriod=2010&endPeriod=2021&dimensionAtObservation=AllDimensions"


response = requests.get(url, headers=headers)
# response = requests.get(url)

with open(f"{OUTPUT_DIR}ERP.csv", "w") as f:
    f.write(response.text)

In [5]:
rsdf = spark.read.csv(f"{OUTPUT_DIR}ERP.csv")
rsdf.show(2, vertical = True, truncate=100)

-RECORD 0---------------------------------
 _c0 | DATAFLOW                           
 _c1 | MEASURE                            
 _c2 | REGION_TYPE                        
 _c3 | ASGS_2021                          
 _c4 | FREQ                               
 _c5 | TIME_PERIOD                        
 _c6 | OBS_VALUE                          
 _c7 | UNIT_MEASURE                       
 _c8 | OBS_STATUS                         
 _c9 | OBS_COMMENT                        
-RECORD 1---------------------------------
 _c0 | ABS:ABS_ANNUAL_ERP_ASGS2021(1.2.0) 
 _c1 | ERP                                
 _c2 | SA2                                
 _c3 | 101021010                          
 _c4 | A                                  
 _c5 | 2010                               
 _c6 | 4813                               
 _c7 | PSNS                               
 _c8 | null                               
 _c9 | null                               
only showing top 2 rows



Household income (weekly) (exclude visitor/non-classifiable) (2021)

In [6]:
url = "https://api.data.abs.gov.au/data/ABS,C21_G33_SA2,1.0.0/...SA2.?startPeriod=2021&dimensionAtObservation=AllDimensions"


response = requests.get(url, headers=headers)

with open(f"{OUTPUT_DIR}Household_income.csv", "w") as f:
    f.write(response.text)

In [7]:
rsdf = spark.read.csv(f"{OUTPUT_DIR}Household_income.csv")
rsdf.show(2, vertical = True, truncate=100)

-RECORD 0---------------------
 _c0 | DATAFLOW               
 _c1 | HIND                   
 _c2 | HHCD                   
 _c3 | REGION                 
 _c4 | REGION_TYPE            
 _c5 | STATE                  
 _c6 | TIME_PERIOD            
 _c7 | OBS_VALUE              
-RECORD 1---------------------
 _c0 | ABS:C21_G33_SA2(1.0.0) 
 _c1 | 7                      
 _c2 | 1_2                    
 _c3 | 101021008              
 _c4 | SA2                    
 _c5 | 1                      
 _c6 | 2021                   
 _c7 | 94                     
only showing top 2 rows



Population projection (2017 - 2066)

In [8]:
url = "https://api.data.abs.gov.au/data/ABS,POP_PROJ_REGION_2012_2061,1.0.0/8+7+6+5+4+3+1+2+11+12+21+22+31+32+41+42+51+52+61+62+71+72.3.TT.1.1.1.1.A?startPeriod=2017&dimensionAtObservation=AllDimensions"


response = requests.get(url, headers=headers)

with open(f"{OUTPUT_DIR}Household_income.csv", "w") as f:
    f.write(response.text)

In [9]:
rsdf = spark.read.csv(f"{OUTPUT_DIR}Household_income.csv")
rsdf.show(2, vertical = True, truncate=100)

-RECORD 0------------------------------------
 _c0  | DATAFLOW                             
 _c1  | REGION                               
 _c2  | SEX_ABS                              
 _c3  | AGE                                  
 _c4  | FERTILITY                            
 _c5  | MORTALITY                            
 _c6  | NOM                                  
 _c7  | NIM                                  
 _c8  | FREQUENCY                            
 _c9  | TIME_PERIOD                          
 _c10 | OBS_VALUE                            
 _c11 | UNIT_MEASURE                         
 _c12 | OBS_STATUS                           
 _c13 | OBS_COMMENT                          
-RECORD 1------------------------------------
 _c0  | ABS:POP_PROJ_REGION_2012_2061(1.0.0) 
 _c1  | 4                                    
 _c2  | 3                                    
 _c3  | TT                                   
 _c4  | 1                                    
 _c5  | 1                         

School location

In [5]:
url = "https://www.acara.edu.au/docs/default-source/default-document-library/school-location-2021e23a2f404c94637ead88ff00003e0139.xlsx?sfvrsn=51ae4c07_0"


response = requests.get(url)

with open(f"{OUTPUT_DIR}School_location.xlsx", "wb") as f:
    f.write(response.content)

# pull_direct(url, OUTPUT_DIR, f"School_location.xlsx")

In [15]:
s_df = pd.read_excel(f"{OUTPUT_DIR}School_location.xlsx", sheet_name="SchoolLocations 2021")
print(s_df.head())

   Calendar Year  ACARA SML ID  Location AGE ID  School AGE ID  \
0           2021         40000      40,003.0000         3.0000   
1           2021         40001      40,004.0000         4.0000   
2           2021         40002      40,005.0000         5.0000   
3           2021         40003      40,007.0000         7.0000   
4           2021         40004      40,009.0000         9.0000   

                                School Name        Suburb State  Postcode  \
0            Corpus Christi Catholic School     BELLERIVE   TAS      7018   
1                              Fahan School     SANDY BAY   TAS      7005   
2                  Geneva Christian College       LATROBE   TAS      7307   
3               Holy Rosary Catholic School     CLAREMONT   TAS      7011   
4  Immaculate Heart of Mary Catholic School  LENAH VALLEY   TAS      7008   

  School Sector School Type  ... Statistical Area 3 Statistical Area 3 Name  \
0      Catholic     Primary  ...              60102     Hobar

In [12]:
type(response.content)

bytes

API test

In [9]:
rsdf = spark.read.csv(f"{OUTPUT_DIR}ABS_C21_G33_SA2_1.0.0_...SA2..csv")
rsdf.show(2, vertical = True, truncate=100)

-RECORD 0------------------------------------
 _c0 | DATAFLOW                              
 _c1 | HIND: Total household income (weekly) 
 _c2 | HHCD: Household composition           
 _c3 | REGION: Region                        
 _c4 | REGION_TYPE: Region Type              
 _c5 | STATE: State                          
 _c6 | TIME_PERIOD: Time Period              
 _c7 | OBS_VALUE                             
-RECORD 1------------------------------------
 _c0 | ABS:C21_G33_SA2(1.0.0)                
 _c1 | 7: $650-$799                          
 _c2 | 1_2: Family households                
 _c3 | 101021008: Karabar                    
 _c4 | SA2: Statistical Area Level 2         
 _c5 | 1: New South Wales                    
 _c6 | 2021                                  
 _c7 | 94                                    
only showing top 2 rows



In [12]:


# https://api.data.abs.gov.au/datastructure/ABS/ALC
# https://api.data.abs.gov.au/datastructure/ABS/ALC?references=codelist
# https://api.data.abs.gov.au/datastructure/ABS/ALC?references=children
# https://api.data.abs.gov.au/data/ALC/1.2.1.4.A.
# https://api.data.abs.gov.au/data/ALC/1.2.1.4.A?startPeriod=2008&endPeriod=2008




# url = "https://api.data.abs.gov.au/data/ABS,C21_G33_SA2,1.0.0/...SA2.?startPeriod=2021&dimensionAtObservation=AllDimensions"
url = "https://api.data.abs.gov.au/datastructure/ABS/ALC"



response = requests.get(url)
print(response.text)

<?xml version="1.0" encoding="utf-8"?>
<!--NSI Web Service v8.8.0.0-->
<message:Structure xmlns:message="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message" xmlns:structure="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure" xmlns:common="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common">
  <message:Header>
    <message:ID>IDREF66461</message:ID>
    <message:Test>false</message:Test>
    <message:Prepared>2022-08-31T01:35:26.2611534+10:00</message:Prepared>
    <message:Sender id="Unknown" />
    <message:Receiver id="Unknown" />
  </message:Header>
  <message:Structures>
    <structure:DataStructures>
      <structure:DataStructure id="ALC" agencyID="ABS" version="1.0.0" isFinal="true">
        <common:Name xml:lang="en">Apparent Consumption of Alcohol, Australia</common:Name>
        <structure:DataStructureComponents>
          <structure:DimensionList id="DimensionDescriptor">
            <structure:Dimension id="TYP" position="1">
              <structure

In [37]:
# .SA2..A?startPeriod=2010&endPeriod=2021&dimensionAtObservation=AllDimensions
temp_sdf = spark.read.csv(f"{OUTPUT_DIR}ABS_ABS_ANNUAL_ERP_ASGS2021_1.2.0_.SA2..A.csv")
temp_sdf.show(2, vertical = True, truncate=100)

-RECORD 0---------------------------------
 _c0 | DATAFLOW                           
 _c1 | MEASURE: Measure                   
 _c2 | REGION_TYPE: Geography Level       
 _c3 | ASGS_2021: Region                  
 _c4 | FREQ: Frequency                    
 _c5 | TIME_PERIOD: Time Period           
 _c6 | OBS_VALUE                          
 _c7 | UNIT_MEASURE: Unit of Measure      
 _c8 | OBS_STATUS: Observation Status     
 _c9 | OBS_COMMENT: Observation Comment   
-RECORD 1---------------------------------
 _c0 | ABS:ABS_ANNUAL_ERP_ASGS2021(1.2.0) 
 _c1 | ERP: Estimated Resident Population 
 _c2 | SA2: Statistical Area Level 2      
 _c3 | 101021010: Queanbeyan - East       
 _c4 | A: Annual                          
 _c5 | 2010                               
 _c6 | 4813                               
 _c7 | PSNS: Persons                      
 _c8 | null                               
 _c9 | null                               
only showing top 2 rows



In [19]:






# try:
#     df.to_csv(cur_dir+"Housing_index.json")
# except Exception as e:
#     print(e)

# cur_df =  pd.json_normalize(response.json(), record_path="results")

# if(cur_df.empty): break
# # start pattern search if required
# if search:
#     cur_df = cur_df.loc[cur_df[search[0]].str.contains(search[1], case=False)]

# # update dataframe
# if(df.empty):
#     if not cur_df.empty:
#         df = cur_df.copy(deep=True)

#         # stop search if greedy True
#         if greedy: return cur_df
# else: df = pd.concat([df, cur_df])

# if save and math.floor(df.shape[0] / save[0]) > 0:
#     print(df.shape[0] % save[0])
#     file_count += 1
#     save_data(df, save[1], save[2]+str(file_count).zfill(3))
#     df = pd.DataFrame(data={"A": []})


TypeError: request() got an unexpected keyword argument 'header'

In [3]:
# print(response)
# print(response.text)
cur_df =  pd.json_normalize(response.json(), record_path="dataSets")

KeyError: "Key 'dataSets' not found. If specifying a record_path, all elements of data should have the path."

In [6]:
boundary_df = pd.read_csv("../data/raw/ABS/Summary of ASGS resources.csv")
boundary_df.head()

,Summary of ASGS resources
Action,ASGS Resource
View and compare boundaries,ABS Maps
View a list of ASGS regions,Allocation files
Convert data from one region to another,Correspondences
View and analyse boundaries in desktop Geospatial Information Systems,Digital boundary files
